In [1]:
from pathlib import Path
desktop_path = Path.home() / "Desktop"
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, rf'{desktop_path}\RA\penicilin\Libaries')

In [2]:
from results_figures import create_results_figures
from sensitivity import perform_sens_uncert_analysis

In [3]:
from reload_lib import reload_lib
import main as m
import results_figures as rfig
import sensitivity as st
import sensitvity_countries as stc
import monte_carlo_energy as mce

In [4]:
reload_lib(m)
init = m.main()

In [5]:
init.bw_project

'Penicillin'

In [6]:
df = init.import_LCIA_results(init.LCIA_results)
df

,"(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, acidification: terrestrial, terrestrial acidification potential (TAP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, climate change, global warming potential (GWP1000))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, ecotoxicity: freshwater, freshwater ecotoxicity potential (FETP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, ecotoxicity: marine, marine ecotoxicity potential (METP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, ecotoxicity: terrestrial, terrestrial ecotoxicity potential (TETP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, energy resources: non-renewable, fossil, fossil fuel potential (FFP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, eutrophication: freshwater, freshwater eutrophication potential (FEP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, eutrophication: marine, marine eutrophication potential (MEP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, human toxicity: carcinogenic, human toxicity potential (HTPc))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, human toxicity: non-carcinogenic, human toxicity potential (HTPnc))",...,"(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, land use, agricultural land occupation (LOP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, material resources: metals/minerals, surplus ore potential (SOP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, ozone depletion, ozone depletion potential (ODPinfinite))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, particulate matter formation, particulate matter formation potential (PMFP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, photochemical oxidant formation: human health, photochemical oxidant formation potential: humans (HOFP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, photochemical oxidant formation: terrestrial ecosystems, photochemical oxidant formation potential: ecosystems (EOFP))","(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, water use, water consumption potential (WCP))","(ReCiPe 2016 v1.03, endpoint (H) - no biogenic, total: ecosystem quality, ecosystem quality)","(ReCiPe 2016 v1.03, endpoint (H) - no biogenic, total: human health, human health)","(ReCiPe 2016 v1.03, endpoint (H) - no biogenic, total: natural resources, natural resources)"
"Penicillin G, defined system",0.002469,0.881954,0.050391,0.070725,2.776495,0.292118,0.000319,0.000097,0.067182,1.022877,...,0.044789,0.017663,2.857530e-07,0.001180,0.001911,0.002042,0.007751,3.941625e-09,2.035841e-06,0.101484
"Penicillin V, defined system",0.000204,0.052935,0.002201,0.002828,0.143012,0.015265,0.000030,0.000006,0.002786,0.054187,...,0.002486,0.000629,3.979416e-08,0.000108,0.000154,0.000157,0.000770,2.571254e-10,1.404129e-07,0.003825


In [7]:
init.bw_project

'Penicillin'

In [ ]:
reload_lib([m, rfig, stc, mce, st])

calc = False
reload = False
sensitivty = False


rfig.create_results_figures(calc=calc, reload=reload) # Calculating the results and creating the figures
st.perform_sens_uncert_analysis(mc_base=10, mc_power=5, reload=reload, calc=calc, sensitivty=sensitivty)
# if calc:
#     init = m.main()
#     dct = init.create_LCI_tables()


In [13]:
reload_lib(mce)
samples = 1000
uncert_func_unit, uncert_dct = mce.obtain_monte_carlo_samples(samples)

Updated 0 exchanges with constructed lognormal uncertainty.
Updated 0 exchanges with constructed lognormal uncertainty.


In [25]:
from bw2data import get_activity
import numpy as np
import pandas as pd

In [14]:
penG = get_activity((init.db.name, "ca75fd45911fc797bc543d0f564fa97d"))
penV = get_activity((init.db.name, "8a5478237433fca863df52bd6c71ffcf"))
penicllin_activities = [
    penG, 
    penV
    ]

In [40]:
uncert_dct = {}
uncert_func_unit = []

for pen in penicllin_activities:
    uncert_dct[pen] = {}
    # uncert_func_unit[pen] = []
    for exc in pen.exchanges():
        if exc["type"] == "technosphere":
            
            s = exc.get('scale')
            l = exc.get('loc')
            # Initialize the key if it does not exist
            penicillin_qunantity = 1
            for up in penG.upstream():
                if up.output != penG:
                    penicillin_qunantity = up["amount"]
            if exc.input not in uncert_dct[pen]:
                uncert_func_unit.append({exc.input : exc["amount"]*penicillin_qunantity})
            uncert_dct[pen][exc.input] = np.random.lognormal(mean=l, sigma=s, size=samples)/exc["amount"]



0.0006


In [142]:
df_uncert = pd.DataFrame(uncert_dct)
df_uncert

,"[production amount, reference product, location, unit, name, worksheet name, database, code]","[production amount, reference product, location, unit, name, worksheet name, database, code]"
"[comment, classifications, activity type, activity, database, filename, location, name, synonyms, parameters, authors, type, reference product, flow, unit, production amount, code]","[1.4816524006643474, 1.1304657544774368, 0.740...","[0.960184911037191, 1.5598209528415852, 1.7574..."
"[comment, classifications, activity type, activity, database, filename, location, name, synonyms, parameters, authors, type, reference product, flow, unit, production amount, code]","[0.5117527619306073, 0.6907417199950294, 2.596...","[1.7536644537475081, 0.8788151204576256, 1.504..."
"[comment, classifications, activity type, activity, database, filename, location, name, synonyms, parameters, authors, type, reference product, flow, unit, production amount, code]","[1.0597326531532074, 1.1557480635352748, 0.964...","[0.8034077448288529, 1.1083130916321868, 0.696..."
"[comment, classifications, activity type, activity, database, filename, location, name, synonyms, parameters, authors, type, reference product, flow, unit, production amount, code]","[0.9177392281107998, 0.9065883121272909, 0.913...","[1.099910180118929, 1.0011234121278187, 0.9459..."
"[comment, classifications, activity type, activity, database, filename, location, name, synonyms, parameters, authors, type, reference product, flow, unit, production amount, code]","[1.0365436191374195, 1.1274792556999982, 0.813...","[0.927932995224469, 0.9810650598958108, 1.2468..."
"[comment, classifications, activity type, activity, database, filename, location, name, synonyms, parameters, authors, type, reference product, flow, unit, production amount, code]","[1.0371274270033173, 0.9437590385572687, 1.089...","[1.095380820939207, 1.0778365854153784, 0.9977..."
"[comment, classifications, activity type, activity, database, filename, location, name, synonyms, parameters, authors, type, reference product, flow, unit, production amount, code]","[0.9997055466516088, 0.9235743798845741, 1.071...","[1.014347834131489, 1.0323732418331881, 1.0523..."
"[production amount, reference product, location, unit, name, worksheet name, database, code]","[2.310275941726857, 1.513403920664029, 3.59406...","[0.8480390705807329, 0.7562258116793479, 0.977..."
"[comment, classifications, activity type, activity, database, filename, location, name, synonyms, parameters, authors, type, reference product, flow, unit, production amount, code]","[24.399891295996156, 12.224733290055543, 4.962...",NaN
"[production amount, reference product, location, unit, name, worksheet name, database, code]","[-0.3014257189700734, -0.6771861904919336, -1....","[-1.8016030994741437, -2.0081731799615135, -2...."


In [41]:
uncert_func_unit

[{'market for acetone, liquid' (kilogram, RoW, None): 0.00013199999999999998},
 {'market for ammonium sulfate' (kilogram, RoW, None): 0.00019199999999999998},
 {'market for butyl acetate' (kilogram, RoW, None): 0.00010799999999999998},
 {'market group for electricity, low voltage' (kilowatt hour, IN, None): 0.013186666666666668},
 {'glucose production' (kilogram, RoW, None): 0.0031079999999999997},
 {'market for heat, from steam, in chemical industry' (megajoule, RoW, None): 0.0040292999999999995},
 {'market for oxygen, liquid' (kilogram, RoW, None): 0.0024119999999999996},
 {'production of pharmamedia' (kilogram, RoW, None): 0.00078},
 {'market for phenyl acetic acid' (kilogram, GLO, None): 1.0718999999999999e-05},
 {'sodium acetate' (kilogram, RoW, None): 0.000156},
 {'market for sodium hydroxide, without water, in 50% solution state' (kilogram, GLO, None): 6.599999999999999e-05},
 {'market for sulfuric acid' (kilogram, RoW, None): 5.999999999999999e-06},
 {'market for water, deionis

In [43]:
import bw2data as bd
import bw2calc as bc

In [44]:
init.database_setup(reload=True)

bd.projects.set_current(init.bw_project)
df = pd.DataFrame(0, index=[list(fu.keys())[0] for fu in uncert_func_unit], columns=[init.lcia_impact_method()[1]], dtype=object)
bd.Database(init.db.name)
# print(f'Total amount of calculations: {len(impact_categories) * len(idx_lst)}')
# # Set up and perform the LCA calculation
calc_setup_name = f'penicillin production uncert'
bd.calculation_setups[calc_setup_name] = {'inv': uncert_func_unit, 'ia': [init.lcia_impact_method()[1]]}
    
mylca = bc.MultiLCA(calc_setup_name)
res = mylca.results

df = pd.DataFrame(res, index=[list(fu.keys())[0] for fu in uncert_func_unit], columns=[init.lcia_impact_method()[1]], dtype=object)


Extracted 1 worksheets in 0.15 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 3.08 seconds
Matching database: ev391cutoff
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


33 datasets
279 exchanges
0 unlinked exchanges
  
Wrote matching file to:
C:\Users\ruw\AppData\Local\pylca\Brightway3\Penicillin.7179b081106185b3e43fbe3ebfd7f229\output\db-matching-penicillin_cut_off_IN-unlinked.xlsx


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 10/01/2025 13:46:37
  Finished: 10/01/2025 13:46:37
  Total time elapsed: 00:00:00
  CPU %: 151.20
  Memory %: 2.32
Created database: penicillin_cut_off_IN


In [70]:
df

,"(ReCiPe 2016 v1.03, midpoint (H) - no biogenic, climate change, global warming potential (GWP1000))"
"market for acetone, liquid",0.000219
market for ammonium sulfate,0.000246
market for butyl acetate,0.000494
"market group for electricity, low voltage",0.019562
glucose production,0.005162
"market for heat, from steam, in chemical industry",0.00052
"market for oxygen, liquid",0.002692
production of pharmamedia,0.000034
market for phenyl acetic acid,0.000062
sodium acetate,0.000372


In [143]:
# df.index = [idx["name"] for idx in df.index]
df_uncert.index = [idx["name"] for idx in df_uncert.index]
df_uncert

,"[production amount, reference product, location, unit, name, worksheet name, database, code]","[production amount, reference product, location, unit, name, worksheet name, database, code]"
"market for acetone, liquid","[1.4816524006643474, 1.1304657544774368, 0.740...","[0.960184911037191, 1.5598209528415852, 1.7574..."
market for ammonium sulfate,"[0.5117527619306073, 0.6907417199950294, 2.596...","[1.7536644537475081, 0.8788151204576256, 1.504..."
market for butyl acetate,"[1.0597326531532074, 1.1557480635352748, 0.964...","[0.8034077448288529, 1.1083130916321868, 0.696..."
"market group for electricity, low voltage","[0.9177392281107998, 0.9065883121272909, 0.913...","[1.099910180118929, 1.0011234121278187, 0.9459..."
glucose production,"[1.0365436191374195, 1.1274792556999982, 0.813...","[0.927932995224469, 0.9810650598958108, 1.2468..."
"market for heat, from steam, in chemical industry","[1.0371274270033173, 0.9437590385572687, 1.089...","[1.095380820939207, 1.0778365854153784, 0.9977..."
"market for oxygen, liquid","[0.9997055466516088, 0.9235743798845741, 1.071...","[1.014347834131489, 1.0323732418331881, 1.0523..."
production of pharmamedia,"[2.310275941726857, 1.513403920664029, 3.59406...","[0.8480390705807329, 0.7562258116793479, 0.977..."
market for phenyl acetic acid,"[24.399891295996156, 12.224733290055543, 4.962...",NaN
sodium acetate,"[-0.3014257189700734, -0.6771861904919336, -1....","[-1.8016030994741437, -2.0081731799615135, -2...."


In [150]:
df_uncert.columns = [col["name"] for col in df_uncert.columns]
df_uncert

,manufacturing of raw penicillium G,manufacturing of raw penicillium V
"market for acetone, liquid","[1.4816524006643474, 1.1304657544774368, 0.740...","[0.960184911037191, 1.5598209528415852, 1.7574..."
market for ammonium sulfate,"[0.5117527619306073, 0.6907417199950294, 2.596...","[1.7536644537475081, 0.8788151204576256, 1.504..."
market for butyl acetate,"[1.0597326531532074, 1.1557480635352748, 0.964...","[0.8034077448288529, 1.1083130916321868, 0.696..."
"market group for electricity, low voltage","[0.9177392281107998, 0.9065883121272909, 0.913...","[1.099910180118929, 1.0011234121278187, 0.9459..."
glucose production,"[1.0365436191374195, 1.1274792556999982, 0.813...","[0.927932995224469, 0.9810650598958108, 1.2468..."
"market for heat, from steam, in chemical industry","[1.0371274270033173, 0.9437590385572687, 1.089...","[1.095380820939207, 1.0778365854153784, 0.9977..."
"market for oxygen, liquid","[0.9997055466516088, 0.9235743798845741, 1.071...","[1.014347834131489, 1.0323732418331881, 1.0523..."
production of pharmamedia,"[2.310275941726857, 1.513403920664029, 3.59406...","[0.8480390705807329, 0.7562258116793479, 0.977..."
market for phenyl acetic acid,"[24.399891295996156, 12.224733290055543, 4.962...",NaN
sodium acetate,"[-0.3014257189700734, -0.6771861904919336, -1....","[-1.8016030994741437, -2.0081731799615135, -2...."


In [173]:
df_impact_sens = pd.DataFrame(None, index=df_uncert.index, columns=df_uncert.columns, dtype=object)

for col in df_uncert.columns:
    for idx, row in df_uncert.iterrows():
        lst = row[col]

        df_impact_sens.at[idx, col] = lst * df.loc[idx].to_numpy()[0]

df_impact_sens

,manufacturing of raw penicillium G,manufacturing of raw penicillium V
"market for acetone, liquid","[0.000325188795914427, 0.00024811136360740197,...","[0.0002107386151673508, 0.0003423450043135713,..."
market for ammonium sulfate,"[0.0001260782770087364, 0.00017017499932288254...","[0.00043204259796439235, 0.0002165098157526832..."
market for butyl acetate,"[0.000523417134059849, 0.0005708405203056406, ...","[0.000396814590952318, 0.0005474117083559022, ..."
"market group for electricity, low voltage","[0.01795284600262182, 0.017734711404788175, 0....","[0.02151648036342535, 0.01958401024716808, 0.0..."
glucose production,"[0.005350726105915232, 0.005820143577143621, 0...","[0.00479006885037747, 0.005064341076118155, 0...."
"market for heat, from steam, in chemical industry","[0.0005397040509177337, 0.0004911166776018106,...","[0.0005700181587778913, 0.0005608884272367787,..."
"market for oxygen, liquid","[0.0026914587537881897, 0.0024864945061479275,...","[0.0027308794741645074, 0.0027794084050200307,..."
production of pharmamedia,"[7.742234522401514e-05, 5.071743971910417e-05,...","[2.841962403715195e-05, 2.5342763088025842e-05..."
market for phenyl acetic acid,"[0.001523399768547055, 0.0007632474931425623, ...",NaN
sodium acetate,"[-0.0001122604743201914, -0.000252205562310545...","[-0.0006709739937744805, -0.000747907227259359..."


In [152]:
mask = df_uncert.notna()
mask

,manufacturing of raw penicillium G,manufacturing of raw penicillium V
"market for acetone, liquid",True,True
market for ammonium sulfate,True,True
market for butyl acetate,True,True
"market group for electricity, low voltage",True,True
glucose production,True,True
"market for heat, from steam, in chemical industry",True,True
"market for oxygen, liquid",True,True
production of pharmamedia,True,True
market for phenyl acetic acid,True,False
sodium acetate,True,True


In [153]:
df_impact = pd.DataFrame(None, index=df_uncert.index, columns=df_uncert.columns, dtype=object)

for col in df_impact.columns:
    for idx, row in df_impact.iterrows():
        row[col] = df.loc[idx].to_numpy()[0]

df_impact = df_impact * mask
df_impact

,manufacturing of raw penicillium G,manufacturing of raw penicillium V
"market for acetone, liquid",0.000219,0.000219
market for ammonium sulfate,0.000246,0.000246
market for butyl acetate,0.000494,0.000494
"market group for electricity, low voltage",0.019562,0.019562
glucose production,0.005162,0.005162
"market for heat, from steam, in chemical industry",0.00052,0.00052
"market for oxygen, liquid",0.002692,0.002692
production of pharmamedia,0.000034,0.000034
market for phenyl acetic acid,0.000062,0.0
sodium acetate,0.000372,0.000372


In [148]:
pen_compact_idx = {
        "Fermentation": ["pharmamedia", "phenyl", "phenoxy", "glucose", "oxygen", "tap water", "sulfate"],
        "Extracion": ["butyl", "sulfuric"],
        "Purification": ["sodium", "acetone"],
        "Energy": ["heat", "electricity"],
        "Waste": ["incineration", "penicillium"]
    }



In [175]:
idx_lst = df_impact.index
lst = [None] * samples
df_categorized = pd.DataFrame(0, index=[key for key in pen_compact_idx.keys()], columns=df_impact_sens.columns, dtype=object)
for col in df_categorized.columns:
    for idx, row in df_categorized.iterrows():
        row[col] = [0] * samples

df_categorized

,manufacturing of raw penicillium G,manufacturing of raw penicillium V
Fermentation,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
Extracion,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
Purification,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
Energy,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
Waste,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [159]:
a = [1] * 10
b = [2] * 10
c = [0] * 10
for i, a_val in enumerate(a): 
    c[i] = a_val + b[i]

c

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [177]:
df_impact_sens

,manufacturing of raw penicillium G,manufacturing of raw penicillium V
"market for acetone, liquid","[0.000325188795914427, 0.00024811136360740197,...","[0.0002107386151673508, 0.0003423450043135713,..."
market for ammonium sulfate,"[0.0001260782770087364, 0.00017017499932288254...","[0.00043204259796439235, 0.0002165098157526832..."
market for butyl acetate,"[0.000523417134059849, 0.0005708405203056406, ...","[0.000396814590952318, 0.0005474117083559022, ..."
"market group for electricity, low voltage","[0.01795284600262182, 0.017734711404788175, 0....","[0.02151648036342535, 0.01958401024716808, 0.0..."
glucose production,"[0.005350726105915232, 0.005820143577143621, 0...","[0.00479006885037747, 0.005064341076118155, 0...."
"market for heat, from steam, in chemical industry","[0.0005397040509177337, 0.0004911166776018106,...","[0.0005700181587778913, 0.0005608884272367787,..."
"market for oxygen, liquid","[0.0026914587537881897, 0.0024864945061479275,...","[0.0027308794741645074, 0.0027794084050200307,..."
production of pharmamedia,"[7.742234522401514e-05, 5.071743971910417e-05,...","[2.841962403715195e-05, 2.5342763088025842e-05..."
market for phenyl acetic acid,"[0.001523399768547055, 0.0007632474931425623, ...",NaN
sodium acetate,"[-0.0001122604743201914, -0.000252205562310545...","[-0.0006709739937744805, -0.000747907227259359..."


In [207]:
for key, st in pen_compact_idx.items():
    for col in df_impact_sens.columns:
        for idx, row in df_impact_sens.iterrows():
            if any(s in idx for s in st):
                lst = row[col]
                # print(lst)
                if lst is not None:
                    try:
                        for i, val in enumerate(lst):
                            df_categorized.at[key, col][i] += val
                    except TypeError as e:
                        print(e)
    df_categorized.at[key, col] = np.array(df_categorized.at[key, col])

# any(s in str(idx) for s in st)
df_categorized

'float' object is not iterable
'float' object is not iterable
'float' object is not iterable
'float' object is not iterable


,manufacturing of raw penicillium G,manufacturing of raw penicillium V
Fermentation,"[0.03907634100193292, 0.037163112061904396, 0....","[0.037608504769753984, 0.03957358175316199, 0...."
Extracion,"[0.002097321387159993, 0.002285235366884151, 0...","[0.001589927214815495, 0.0021904919230427066, ..."
Purification,"[0.0011401493854326845, 0.0014785454989709474,...","[-0.0017483311737117254, -0.001549263443024551..."
Energy,"[0.0739702002141582, 0.07290331232955993, 0.07...","[0.08834599408881295, 0.08057959469761944, 0.0..."
Waste,"[0.006266939481868471, 0.005363730416877759, 0...","[0.005619312502456874, 0.006242016241988317, 0..."


In [218]:
x = df_categorized.at[key, col]
x.std()

0.0006241836555884501

In [ ]:
x.min()

0.003543621314119841

In [220]:
x.mean()

0.005599291146112177

In [197]:
from scipy import stats
import matplotlib.pyplot as plt


In [ ]:
mins = x.min()
mins

0.003543621314119841

In [202]:
x

array([[ 0.12573022, -0.13210486,  0.64042265, ...,  0.36159505,
         1.30400005,  0.94708096],
       [-0.70373524, -1.26542147, -0.62327446, ..., -0.21879166,
        -1.24591095, -0.73226735],
       [-0.54425898, -0.31630016,  0.41163054, ...,  1.36646347,
        -0.66519467,  0.35151007],
       ...,
       [-0.15151926, -1.56936813, -2.37558809, ..., -1.25556531,
         2.26383369,  0.79665938],
       [-0.40002898, -0.5131527 ,  1.19950067, ...,  1.71448637,
         1.04121411,  0.10901068],
       [-0.0518811 ,  1.61830256,  1.00300418, ...,  0.66987823,
         0.94582488,  0.43950909]])